In [21]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder
from msgraph.generated.search.query.query_post_request_body import QueryPostRequestBody
from msgraph.generated.search.query.query_request_builder import QueryRequestBuilder
from msgraph.generated.search.search_request_builder import SearchRequestBuilder
from msgraph.generated.models.search_request import SearchRequest
from msgraph.generated.models.entity_type import EntityType
from msgraph.generated.models.search_query import SearchQuery
from msgraph.generated.models.search_response import SearchResponse
import os
import json
import asyncio 

In [2]:
load_dotenv()

True

In [3]:
TENANT_ID = os.getenv("TENANT_ID")

In [4]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [5]:
graphUserScopes = os.getenv("graphUserScopes").split(' ')

In [6]:
print(CLIENT_ID)

48943d42-a075-43c5-8d93-3f2da879c1be


In [7]:
fabs = 'fabian@fabster.onmicrosoft.com'

In [8]:
adele = 'adelev@fabster.onmicrosoft.com'

In [9]:
## ensure that you have turned on DefaultClientType to Yes in the App Registration

In [10]:
print(graphUserScopes)

['User.Read', 'Mail.ReadWrite', 'Chat.ReadWrite', 'Calendars.Read', 'Files.Read.All', 'Sites.Read.All']


In [11]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [13]:
result = await client.me.get()

In [14]:
print(result.display_name)

Adele Vance


In [ ]:
async def get_user(currUser):
    result = await client.users.by_user_id(currUser).get()
    if result:
        print(result.display_name)

In [ ]:
await (get_user('adelev@fabster.onmicrosoft.com'))

In [ ]:
async def get_user_messages(currUser):
    messages = await (client.users.by_user_id(currUser).messages.get())
    if messages and messages.value:
        for msg in messages.value:
            print(msg.subject, msg.id, msg.from_, msg.body_preview)

In [ ]:
await get_user_messages(adele)

In [ ]:
async def get_user_chats(currUser):
    chats = await (client.users.by_user_id(currUser).chats.get())
    if chats and chats.value:
        for cht in chats.value:
            print(cht.topic)

In [ ]:
await get_user_chats(adele)

In [52]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.ChatMessage
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

chat_result = await client.search.query.post(request_body)

In [53]:
chat_data = chat_result.value

In [54]:
chat_data

[SearchResponse(additional_data={}, hits_containers=[SearchHitsContainer(additional_data={}, aggregations=None, hits=[SearchHit(additional_data={}, content_source=None, hit_id='AQMkADQ1ZjQzNTE5LTRjNGItNDI1ZC1hNWY1LWNmZTEzNjBjMGY3MgBGAAADTsefQxogGUq/s09IsB8ApgcAR7eBlyFi6U2kmUMoQtaH+AAAAoIEAAAAR7eBlyFi6U2kmUMoQtaH+AACwsQOtAAAAA==', is_collapsed=None, odata_type=None, rank=1, resource=Entity(additional_data={'createdDateTime': DateTime(2023, 11, 23, 17, 52, 21, tzinfo=Timezone('UTC')), 'lastModifiedDateTime': DateTime(2023, 11, 23, 18, 0, 40, tzinfo=Timezone('UTC')), 'subject': '', 'importance': 'normal', 'webLink': 'https://teams.microsoft.com/l/message/19%3a28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6%40unq.gbl.spaces/1700761940682?context=%7B%22contextType%22:%22chat%22%7D', 'from': {'emailAddress': {'name': 'Adele Vance', 'address': 'AdeleV@fabster.onmicrosoft.com'}}, 'channelIdentity': {'channelId': '19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-4

In [90]:
search_response = chat_data[0]  # Assuming there's only one SearchResponse object in the list

# Initialize an empty list to store summaries and chat IDs
summaries_and_chat_ids = []

# Iterate over hits_containers in the SearchResponse object
for hits_container in search_response.hits_containers:
    # Iterate over each hit in the container
    for hit in hits_container.hits:
        # Extract the summary
        summary = hit.summary

        # Extract the chatId from additional_data
        chat_id = hit.resource.additional_data.get('chatId', 'Unknown')

        # Append the chatId and summary as a tuple to the list
        summaries_and_chat_ids.append((chat_id, summary))

# Now 'summaries_and_chat_ids' contains the chatId and summary of each search hit
for item in summaries_and_chat_ids:
    print(item)

('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hmmm which one? the one closer to the book store by Rokin near the Yo boba store?')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hey Im in Amsterdam downtown lets get some Fries from that spot near Amsterdam Central next to the Uni Clo store')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/personal/fabian_fabster_onmicrosoft_com/EfYFSS6IwSZNnyCyYPcK4IkBRiFeuXhA3cHn74Ha6Ag_Ow https://fabster-my.sharepoint.c')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'how are you...')
('19:28031fbd-75af-4b61-8afd-347aac675901_eb14fa25-43e5-45f8-973d-7ea79a346db6@unq.gbl.spaces', 'hi adele long time')
('19:28031fbd-75af-4

In [ ]:
print(chat_summary_and_ids)

In [76]:
try:
    chat_search_response = chat_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries and from information
    chat_summary_and_from = []

    # Iterate over hits_containers in the SearchResponse object
    for chat_hits_container in chat_search_response.hits_containers:
        # Iterate over each hit in the container
        for chat_hit in chat_hits_container.hits:
            # Extract the summary
            chat_summary = chat_hit.summary

            # Extract the 'from' information from additional_data
            from_data = chat_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
            from_name = from_data.get('name', 'Unknown')
            from_email = from_data.get('address', 'Unknown')

            # Append the summary and 'from' information as a tuple to the list
            chat_summary_and_from.append((chat_summary, from_name, from_email))

except Exception as e:
    print(f"An error occurred: {e}")
    # Optionally, you can exit the script or function here
    # exit()

'''
# Now 'summary_and_from' contains the summary and from information of each search hit
for item in chat_summary_and_from:
    print(item)
'''

"\n# Now 'summary_and_from' contains the summary and from information of each search hit\nfor item in chat_summary_and_from:\n    print(item)\n"

In [77]:
print(chat_summary_and_from)

[('hmmm which one? the one closer to the book store by Rokin near the Yo boba store?', 'Adele Vance', 'AdeleV@fabster.onmicrosoft.com'), ('hey Im in Amsterdam downtown lets get some Fries from that spot near Amsterdam Central next to the Uni Clo store', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/personal/fabian_fabster_onmicrosoft_com/EfYFSS6IwSZNnyCyYPcK4IkBRiFeuXhA3cHn74Ha6Ag_Ow https://fabster-my.sharepoint.c', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('how are you...', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('hi adele long time', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('u there...', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('cool well i am going to test now on Postman...', 'Fabian Williams', 'fabian@fabster.onmicrosoft.com'), ('i want to have a chat before my status change...', '

## Work on Drives

In [79]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.DriveItem
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

drive_result = await client.search.query.post(request_body)

In [80]:
drive_data = drive_result.value

In [81]:
try:
    drive_search_response = drive_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries and from information
    drive_summary_and_from = []

    if drive_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for drive_hits_container in drive_search_response.hits_containers:
            # Iterate over each hit in the container
            for drive_hit in drive_hits_container.hits:
                # Extract the summary
                drive_summary = drive_hit.summary

                # Extract the 'from' information from additional_data
                from_data = drive_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Append the summary and 'from' information as a tuple to the list
                drive_summary_and_from.append((drive_summary, from_name, from_email))
except Exception as e:
    print(f"An error occurred: {e}")

In [82]:
print(drive_summary_and_from)

[('', 'Unknown', 'Unknown'), ('Project Roundtree Status Report Everything is going fine. Customer paid a ton of money<ddd/>', 'Unknown', 'Unknown'), ('<ddd/><c0>Adele</c0> Vance 1 856 0 {FABE1924<ddd/>', 'Unknown', 'Unknown'), ('<ddd/><c0>Adele</c0> Vance 1 198 0 {5CBA3280<ddd/>', 'Unknown', 'Unknown'), (' dynamic externalitem = new JObject(); externalitem.acl = new JArray{ new JObject{{"type", "user" }, {"type", "bac58813-8954-4633-9503-cc51c0ab34c9"}, { "accessType", "grant<ddd/>', 'Unknown', 'Unknown'), ('<ddd/><c0>Adele</c0> Vance 1 0 {D5FE893F-94AC<ddd/>', 'Unknown', 'Unknown')]


## Work on Mail

In [85]:
request_body = QueryPostRequestBody(
	requests = [
		SearchRequest(
			entity_types = [
				EntityType.Message
			],
			query = SearchQuery(
				query_string = "Adele",
			),
		),
	],
)

mail_result = await client.search.query.post(request_body)

In [86]:
mail_data = mail_result.value

In [87]:
try:
    mail_search_response = mail_data[0]  # Assuming there's only one SearchResponse object in the list

    # Initialize an empty list to store summaries and from information
    mail_summary_and_from = []

    if mail_search_response:
        # Iterate over hits_containers in the SearchResponse object
        for mail_hits_container in mail_search_response.hits_containers:
            # Iterate over each hit in the container
            for mail_hit in mail_hits_container.hits:
                # Extract the summary
                mail_summary = mail_hit.summary

                # Extract the 'from' information from additional_data
                from_data = mail_hit.resource.additional_data.get('from', {}).get('emailAddress', {})
                from_name = from_data.get('name', 'Unknown')
                from_email = from_data.get('address', 'Unknown')

                # Append the summary and 'from' information as a tuple to the list
                mail_summary_and_from.append((mail_summary, from_name, from_email))
except Exception as e:
    print(f"An error occurred: {e}")

In [88]:
print(mail_summary_and_from)

[('Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Explore your insights  27 days without quiet...', 'Unknown', 'Unknown'), ('Private to you   Hi, <c0>Adele</c0> Vance,  Discover trends in your work habits  An in-depth look at your work patterns in the last four weeks  Work smarter with Microsoft Viva Insights  ...', 'Unknown', 'Unknown'), ("Fabian Williams shared this with you 7 days ago. If you missed it, here's a quick reminder  This link will work for anyone.  TeachKalebHowToRideABicycle  Open   Microsoft respects...", 'Unknown', 'Unknown'), ("Fabian Williams shared this with you 7 days ago. If you missed it, here's a quick reminder  This link will work for anyone.  MariamAssignment2DraftDayCare  Open   Microsoft respects...", 'Unknown', 'Unknown'), ('I need to fix Kaleb Williams bicycle today What tools should i bring based on this document here https://fabster-my.sharepoint.com/:w:/g/p